In [ ]:
import os
import sqlite3
import PyPDF2
import fitz
import PyPDF2.errors
import io
from PIL import Image
from pytesseract import image_to_string

In [ ]:
# Criação do banco
con = sqlite3.connect("database/documentos.db")

cursor = con.cursor()
    
# Cria a tabela se não existir
cursor.execute("""
CREATE TABLE IF NOT EXISTS ocr_results (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_name TEXT NOT NULL,
    nr_processo TEXT NOT NULL,
    ocr_text TEXT NOT NULL,
    tipo_doc TEXT NOT NULL
);
""")
con.commit()
con.close()

In [ ]:
def is_text_pdf(file_path: str):
    """
    Verifica se um arquivo PDF contém texto extraível.

    Parameters:
    ----------
    file_path : str
        Caminho para o arquivo PDF a ser analisado.

    Returns:
    -------
    bool
        Retorna True se o PDF contiver pelo menos uma página com texto extraível.
        Caso contrário, retorna False.
    """
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                if page.extract_text():
                    return True
            return False
    except PyPDF2.errors.PdfReadError:
        print(f"Erro ao ler o arquivo PDF: {file_path}. O arquivo pode estar corrompido.")
    except Exception as e:
        print(f"Erro inesperado ao processar o arquivo {file_path}: {e}")
                
    return None

def extract_text_from_pdf(file_path: str):
    """
    Extrai o texto de todas as páginas de um arquivo PDF.

    Parameters:
    ----------
    file_path : str
        Caminho para o arquivo PDF de onde o texto será extraído.

    Returns:
    -------
    str
        Uma string contendo todo o texto extraído do PDF, concatenado de todas as páginas.
    """
    text = ""
    with fitz.open(file_path) as pdf:
        # Itera por todas as páginas e extrai o texto
        for page in pdf:
            text += page.get_text()
    return text

def extract_images_from_pdf(file_path: str):
    """
    Extrai todas as imagens de um arquivo PDF.

    Parameters:
    ----------
    file_path : str
        Caminho para o arquivo PDF de onde as imagens serão extraídas.

    Returns:
    -------
    list
        Uma lista de bytes representando as imagens extraídas do PDF.
    """
    doc = fitz.open(file_path)
    images = []
    for page_num in range(len(doc)):
        for img_index, img in enumerate(doc[page_num].get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            images.append(image_bytes)
    return images

def extract_text_from_images(images):
    """
    Extrai o texto de uma lista de imagens usando OCR.

    Parameters:
    ----------
    images : list
        Lista de bytes representando imagens de onde o texto será extraído.

    Returns:
    -------
    str
        Um único texto concatenado extraído de todas as imagens.
    """
    texts = []
    for image_bytes in images:
        image = Image.open(io.BytesIO(image_bytes))
        text = image_to_string(image, lang='por')
        texts.append(text)
    return ''.join(texts)

def insert_ocr_result(file_name: str, nr_processo: str, ocr_text: str, tipo_doc: str):
    """
    Insere o resultado do OCR no banco de dados.

    Parameters:
    ----------
    file_name : str
        Nome do arquivo PDF processado.
    file_name : str
        Número do processo PDF processado.
    ocr_text : str
        Texto extraído do arquivo PDF.
    tipo_doc : str
        Documento foi imagem ou texto
    """
    con = sqlite3.connect("database/documentos.db")
    cursor = con.cursor()
    
    try:
        cursor.execute("""
            INSERT INTO ocr_results (file_name, nr_processo, ocr_text, tipo_doc)
            VALUES (?, ?, ?, ?)
        """, (file_name, nr_processo, ocr_text, tipo_doc))
        con.commit()
    except sqlite3.Error as e:
        print(f"Erro ao inserir no banco de dados: {e}")
    finally:
        con.close()

In [ ]:
# Caminho da pasta onde estão os PDFs
folder_path = "pdfs"
# Percorrendo todos os arquivos na pasta
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".pdf"):  # Verifica se o arquivo é um PDF
            file_path = os.path.join(root, file)
            pdf_text = is_text_pdf(file_path)
                        
            if pdf_text is True:
                text = extract_text_from_pdf(file_path)
                tipo_doc = 'text'
            elif pdf_text is False:
                images = extract_images_from_pdf(file_path)
                text = extract_text_from_images(images)
                tipo_doc = 'image'
            else:
                print('PDF Inválido')
                continue
            
            nr_processo = file.split('.pdf')
            
            insert_ocr_result(file, nr_processo[0], text, tipo_doc)
            print(f'Documento {file_path} inserido no banco')